In [1]:
import os 
import tensorflow as tf
import math
import numpy as np 
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory

/home/sardor/PycharmProjects/ColorIdentificationinImages/venv/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/home/sardor/PycharmProjects/ColorIdentificationinImages/venv/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/home/sardor/PycharmProjects/ColorIdentificationinImages/venv/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/home/sardor/PycharmProjects/ColorIdentificationinImages/venv/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/home/sardor/PycharmProjects/ColorIdentificationinImages/venv/lib/pyth

In [2]:
IMG_SIZE=128
BATCH_SIZE=64 # assume kaggle know something
SEED=17
EPOCHS=10
TRAIN_DIR = "mask_images/train"
TEST_DIR = "mask_images/test"
VAL_DIR = "mask_images/val"

In [3]:
train_gen = image_dataset_from_directory(
    TRAIN_DIR,
    label_mode='binary',
    image_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    interpolation='nearest',
    labels='inferred', #i also do not know why i am adding them 
    shuffle=True
)

test_gen = image_dataset_from_directory(
    TEST_DIR,
    image_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    label_mode='binary',
    labels='inferred',
    interpolation='nearest')

val_gen = image_dataset_from_directory(
    VAL_DIR,
    image_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    labels='inferred',
    label_mode='binary',
    interpolation='nearest'
)


Found 72 files belonging to 2 classes.
Found 14 files belonging to 2 classes.
Found 14 files belonging to 2 classes.


In [4]:
#data pipeline 
def convert_to_float(image,label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image,label


AUTOTUNE = tf.data.experimental.AUTOTUNE

ds_train = train_gen.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)
ds_val = test_gen.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)
ds_test = val_gen.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)

In [5]:
pretrained_base = tf.keras.models.load_model("saved_models/vgg16-pretrained-base")
pretrained_base.trainable=False


2022-12-17 16:03:03.080805: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 9437184 exceeds 10% of free system memory.
2022-12-17 16:03:03.100492: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 9437184 exceeds 10% of free system memory.
2022-12-17 16:03:03.113072: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 9437184 exceeds 10% of free system memory.
2022-12-17 16:03:03.150280: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 9437184 exceeds 10% of free system memory.
2022-12-17 16:03:03.174450: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 9437184 exceeds 10% of free system memory.


In [8]:
from tensorflow.keras.models import Sequential


model=Sequential([
    pretrained_base,
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dense(1, activation='sigmoid')])


model.compile(loss='binary_crossentropy',  
             optimizer='adam',
             metrics=['binary_accuracy'])


In [9]:
history = model.fit(ds_train, epochs=EPOCHS, 
                    validation_data=ds_val)

Epoch 1/10
2/2 [==============================] - 24s 7s/step - loss: 0.9363 - binary_accuracy: 0.3611 - val_loss: 0.4538 - val_binary_accuracy: 0.8571
Epoch 2/10
2/2 [==============================] - 17s 4s/step - loss: 0.4307 - binary_accuracy: 0.8194 - val_loss: 0.4238 - val_binary_accuracy: 0.7143
Epoch 3/10
2/2 [==============================] - 13s 3s/step - loss: 0.4206 - binary_accuracy: 0.7222 - val_loss: 0.3430 - val_binary_accuracy: 0.8571
Epoch 4/10
2/2 [==============================] - 13s 3s/step - loss: 0.3289 - binary_accuracy: 0.8750 - val_loss: 0.1556 - val_binary_accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 13s 3s/step - loss: 0.1532 - binary_accuracy: 1.0000 - val_loss: 0.0770 - val_binary_accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 13s 3s/step - loss: 0.0885 - binary_accuracy: 1.0000 - val_loss: 0.0609 - val_binary_accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 14s 3s/step - loss: 0.0771 - binary_ac

In [10]:
model.evaluate(ds_test, verbose=2)
model.evaluate(ds_val, verbose=2)

1/1 - 2s - loss: 0.0389 - binary_accuracy: 1.0000 - 2s/epoch - 2s/step
1/1 - 3s - loss: 0.0219 - binary_accuracy: 1.0000 - 3s/epoch - 3s/step


[0.021900800988078117, 1.0]

In [11]:
model.save('saved_models/my_model_kaggle_course_v3')

INFO:tensorflow:Assets written to: saved_models/my_model_kaggle_course_v3/assets


INFO:tensorflow:Assets written to: saved_models/my_model_kaggle_course_v3/assets
